<table border="0" width="100%"><p align="left"><img src="https://fic.swufe.edu.cn/img/logo_2.png"  align="left" width=30%></table>

# <center>Chapter 6 循环神经网络</center>

**&ensp; &ensp; 在前面的章节中，我们学会了利用某种模型例如word2vec对文本数据进行建模，那么现在我们来深入探讨文本数据的一些特征：对于一个文本数据而言，本质上它是一个序列，对于一些序列输入的信息(文本、语音)，不同时刻之间的输入存在相互的影响，我们阅读文本总是从左到右依次的阅读，也会记忆之前阅读的文字，方便我们理解全文。那么对于计算机而言，也需要一种模型能够“记忆”历史输入的信息，进而对整个序列进行完整的特征提取和表征。
循环神经网络(RNN)就是面对这样的需求提出来的，它能够“记忆”序列输入的历史信息，从而能够较好的对整个序列进行语义建模。**

**&ensp; &ensp; 在金融领域，循环神经网络也是非常重要的一类模型，因为我们的大部分金融数据，例如股价，公司市场信息等也是存在时间序列的属性，可以当成一个序列来建模。掌握循环神经网络的原理和应用对于我们学习文本数据建模和金融场景数据建模十分重要。**
###  本章节基本组织如下:
* RNN
* LSTM
* 利用循环神经网络实现文本生成

## 1.RNN模型

### 1.1 RNN基本原理

&ensp; &ensp; 一般的神经网络都是单向的，一层连着下一层。而**循环神经网络（Recurrent Neural Network）**和它的名字一样，里面引入了**循环体结构**，就像我们写代码的`for`或者`while`循环一样，某一步的循环体就像下面这样：

<center><img src="https://img-blog.csdnimg.cn/20200601195248631.png" width=200px><center>

&ensp; &ensp;$x_t$是第$t$步循环时的输入，$h_t$是第$t$步循环的输出，$h_t$也被称为$t$时刻的**隐藏状态$hidden$**，它们都是向量，不是标量。这样一个循环体就可以把信息从上一步传递到下一步。不过，这样的循环体看起来不太好懂，让我们把它按时序展平，变成一般的神经网络那样的单向传播结构。展开后就是一个链状结构：

<div style="width:60%;height:60%;margin:0 auto;">
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png"/>
</div>

&ensp; &ensp;举一个简单的文本例子：假设我们输入的序列为**“我 爱 西 财”**这一句话，$x_0$是第$0$步循环时的输入，也就是“我”字，$h_0$为一个包含了“我”字向量信息的输出，那么$x_1$是第$1$步循环时的输入，也就是“爱”字；$h_1$不仅包含了“爱”字向量信息，也包含了上一步“我”字的向量信息；同理，依次循环，最后的$h_3$其实是一个包含“我 爱 西 财”向量信息的输出，这样就实现了序列的**“记忆”**。

&ensp; &ensp;用数学公式表示为：

![RNN公式](RNN公式.png)

&ensp; &ensp;其中$W$为权重矩阵，$b$为偏置，都为**模型参数**，也是后续训练优化的对象；$tanh$为**激活函数**，也可以使用$relu$等其他激活函数。
以上就是循环神经网络RNN最基本的一个**循环单元**，现在我们利用上面“我爱西财”的例子，用纯python的代码去解析它。假设不存在偏置项的情况下，那么首先我们定义了一个方法`RNNstep`，来完成上面公式的计算，即RNN循环的一步。

### 1.2 RNN基本代码实现(拆分版)

&ensp; &ensp;RNN的代码本质上就是在一个普通的神经网络基础上，多加一个**hidden_state**来保存历史信息。用公式来表示的话就是：
$h_t=tanh(w_h$$x_t+b_t+w_h+b_h)$,而普通的神经网络只有$w_h$$x_t+b_h$，少了隐藏状态的信息。那么我们可以通过构建一个`RNNstep`方法来存储隐藏状态信息：

In [1]:
import numpy as np

def RNNstep(x, hidden):
    # 更新隐藏状态
    hidden = np.tanh(np.dot(W_hh, hidden) + np.dot(W_xh, x))   # 注意看，其实实现的就是上面的公式(无偏置项)
    return hidden  # 当前inpu和隐藏状态作为输入，得到下一步的隐藏状态

给定我们的输入文本数据**“我 爱 西 财”**，这里作一个最简单的onehot处理：

In [2]:
# 我 爱 西 财
text_data = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
print(f"输入的向量形状为:{text_data.shape}")  #  [seq_len * input_size]
seq_len = text_data.shape[1]
print(f"句子长度为:{seq_len}")

输入的向量形状为:(4, 4)
句子长度为:4


从第0步开始，我们需要输入“我”字($x_0$ = [1,0,0,0]),此时不存在**hidden_state**，也就是$h_0$，可以手动随机初始化的hidden_state；除此之外，我们还应该初始化参数，这里为$W$：

In [3]:
hidden_state = np.random.random((4,4))
 # 这里的hidden_state的维度设置本质上为序列长度，即循环输入时应该用到多少个隐藏单元
print(f"初始隐藏状态:\n{hidden_state}")

W_hh = W_xh = np.random.random((4,4))
print(f"初始权重矩阵:\n{W_hh}")

初始隐藏状态:
[[0.23615902 0.44332739 0.52193922 0.46863795]
 [0.74552592 0.44449781 0.97458196 0.29072798]
 [0.18527842 0.31399347 0.97185445 0.04075662]
 [0.1519588  0.88534927 0.49714817 0.85433517]]
初始权重矩阵:
[[0.37018877 0.9345049  0.43001064 0.3104593 ]
 [0.16109335 0.11693149 0.21544091 0.77692601]
 [0.58470632 0.66477251 0.98469315 0.07241005]
 [0.90195891 0.18030394 0.05723204 0.42469328]]


一共4个字，那么我们应该循环计算4步，最终得到的**hidden_state**为最后的隐藏状态$h_3$。

In [4]:
for i in range(seq_len):
    hidden_state = RNNstep(text_data[i,:], hidden_state) 
print(f"最终隐藏状态:\n{hidden_state}")

最终隐藏状态:
[[0.97885185 0.9902648  0.96978843 0.97910535]
 [0.91175181 0.96153125 0.86889499 0.92129702]
 [0.9878806  0.99458351 0.98220887 0.98853489]
 [0.94988961 0.97819583 0.92491659 0.95506927]]


RNN的循环单元部分比较抽象，不过也是最重要的部分，让我们再回顾以下流程：

<div style="width:680px;height:350px;margin:0 auto;">
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png"/>
</div>

![RNN流程示意图](Rnn_formula.png)

### 1.3利用Pytorch实现RNN模型

上面已经介绍了RNN的基本源码，让我们使用pytorch的RNN模块来快速搭建循环神经网络：

In [5]:
import torch
import torch.nn as nn

定义`exampleRNN`，输入为**[input_size :输入特征维数（特征向量的长度）
;hidden_size:隐层状态的维数]：**

In [6]:
class exampleRNN(object):
    def __init__(self,input_size,hidden_size):
        super().__init__()
        self.W_xh = torch.nn.Linear(input_size, hidden_size) #因为最后的操作是相加 所以hidden要和output的shape一致
        self.W_hh = torch.nn.Linear(hidden_size, hidden_size)

    def __call__(self, x, hidden):
        return self.step(x, hidden)
    
    def step(self, x, hidden):
        #前向传播的一步
        h1 = self.W_hh(hidden)
        w1 = self.W_xh(x)
        out = torch.tanh(h1 + w1)
        hidden = self.W_hh.weight #隐藏状态权重
        return out, hidden

给定input_text(我 爱 西 财)，利用`exampleRNN`建立rnn模型：

In [7]:
input_text = torch.from_numpy(text_data).float()  # [seq_len * input_size],从numpy转化为tensor.float格式
rnn = exampleRNN(4, 4)   #[input_size*hidden_size]  
h_0 = torch.randn(4, 4)   # [seq_len*hidden_size]
seq_len = input_text.shape[0]  # 4，句子长度

注：因为文本向量one_hot输入的形状是4*4，且只有一句话，没有考虑batch_size，所以这里input_size、hidden_size、seq_len都为4，容易混淆原本的维度含义，请参照注释理解

循环得到最终输出

In [8]:
for i in range(seq_len):  #循环计算句子长度seq_len次
    output, hn = rnn(input_text[i, :], h_0)
print(output.size(), output)  

torch.Size([4, 4]) tensor([[ 0.1478,  0.3786,  0.4213, -0.3913],
        [ 0.1763,  0.7221, -0.8491, -0.4117],
        [-0.0558,  0.4432,  0.3692,  0.1021],
        [-0.2990,  0.5833, -0.0143,  0.2127]], grad_fn=<TanhBackward0>)


### 1.4直接通过Pytorch自带的模块实现RNN模型

事实上，`pytorch`定义了完整的RNN层级模块，可以直接进行调用，详细源码和参数设置可以参照[pytorch_RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html?highlight=rnn#torch.nn.RNN)：

In [9]:
input_text = torch.from_numpy(text_data[np.newaxis, :]).float()  # np.newaxis矩阵扩充维度，即加入新维度，batch_size = 1
print(f"输入的向量形状为:{input_text.shape}")
rnn = nn.RNN(4,4,1)  # 同理，在pttorch自带的rnn模块里面，也是引入了batch_size这一维度
h0 = torch.randn(1,4,4)

output, hn = rnn(input_text, h0)
print(output.size(), output)

输入的向量形状为:torch.Size([1, 4, 4])
torch.Size([1, 4, 4]) tensor([[[ 0.3493, -0.2748, -0.4350, -0.5564],
         [ 0.9444,  0.4666,  0.2933,  0.8598],
         [-0.1196, -0.6185,  0.0732,  0.4923],
         [ 0.9380,  0.5048, -0.5799, -0.7094]]], grad_fn=<StackBackward0>)


## 2.LSTM模型

### 2.1 RNN 模型中存在的缺陷

&ensp; &ensp;通过学习之前的知识，我们了解到**RNN(Recurrent Neural Network 循环神经网络)** 模型在处理**序列场景**中拥有较好的表现，即前面某一时刻的信息会对后面某个时刻的产生影响，例如在文本预测中，通过前面的信息预测后文中的某一个词的场景。但是RNN模型面临当**序列长度较大**的时候，在训练模型的过程中很容易出现**梯度消失/梯度爆炸**的问题，**即后面某一时刻无法接收到前面某个时刻传递过来的文本权重信息**，使得权重矩阵的更新出现问题，进而使得模型训练很难到达理想的效果。

### 2.1.1 梯度消失/梯度爆炸
&ensp; &ensp;**梯度消失**：在反向传播过程中，靠后的网络层的权重更新需要使用到靠前的网络层的权重偏导信息，越靠后的网络层能够计算关于权重矩阵正常的偏导数，而越靠近输入层的网络层计算得到的偏导数近乎为0，于是越靠后的网络层的梯度更新将基本上无变化，称为梯度消失。<br/>
&ensp; &ensp;**梯度爆炸**：同理，越靠后的网络层权重矩阵更新会使用前面网络层权重信息，如果靠近输入层的网络层的梯度信息变得极大，那么更新后的权重信息也会变得极大，称为梯度爆炸<br/>
&ensp; &ensp;**为了更好的理解梯度消失和梯度爆炸，将通过下面的例子进行说明：**

![](https://pic2.zhimg.com/80/v2-37ba7e208c51e0a5bfd37b60da074b79_720w.webp)

&ensp; &ensp;上图中是一个标准的RNN模型，Xt表示在t时刻的输入，Wx表示输入的权重矩阵，Ws表示隐藏层上一时刻传递的权重矩阵，Wo表示隐藏层到输出层的权重矩阵，Ot表示t时刻的输出。假设我们的序列只有三段：s1、s2、s3。其中s0表示的是一个固定值，使用的激活函数为tanh,则RNN的前向传播过程如下：

\begin{align}
\mathrm{S}_1&= tanh(W_xX_1 + W_sS_0 + b_1)\\  
\mathrm{S}_2&= tanh(W_xX_2 + W_sS_1 + b_1)\\
\mathrm{S}_3&= tanh(W_xX_3 + W_sS_2 + b_1)\\
\mathrm{O}_1&= W_oS_1 + b_2\\
\mathrm{O}_2&= W_oS_2 + b_2\\
\mathrm{O}_3&= W_oS_3 + b_2\\
\end{align}

&ensp; &ensp;假设在t = 3的时候，我们的损失函数为$L_3 = \frac{1}{2}(Y_3 - O_3)^2$,使用梯度下降法求解$L_3$的最小值的过程就是对$W_x,W_s,W_o$求偏导，不断的调整三个权重的值，然后使得$L_3$变得最小，下面列出对$W_o,W_x,W_s$的求偏导过程：

\begin{align}
\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{W_o}}}&=\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{O_3}}}\frac{\partial{\mathrm{O_3}}}{\partial{\mathrm{W_o}}}\\
\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{W_x}}}&=\frac{\partial{\mathrm{L_3}}}
{\partial{\mathrm{O_3}}}\frac{\partial{\mathrm{O_3}}}{\partial{\mathrm{S_3}}}\frac{\partial{\mathrm{S_3}}}{\partial{\mathrm{W_x}}}+
\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{O_3}}}\frac{\partial{\mathrm{O_3}}}{\partial{\mathrm{S_3}}}\frac{\partial{\mathrm{S_3}}}{\partial{\mathrm{S_2}}}\frac{\partial{\mathrm{S_2}}}{\partial{\mathrm{W_x}}}+
\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{O_3}}}\frac{\partial{\mathrm{O_3}}}{\partial{\mathrm{S_3}}}\frac{\partial{\mathrm{S_3}}}{\partial{\mathrm{S_2}}}\frac{\partial{\mathrm{S_2}}}{\partial{\mathrm{S_1}}}\frac{\partial{\mathrm{S_1}}}{\partial{\mathrm{W_x}}}\\
\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{W_s}}}&=\frac{\partial{\mathrm{L_3}}}
{\partial{\mathrm{O_3}}}\frac{\partial{\mathrm{O_3}}}{\partial{\mathrm{S_3}}}\frac{\partial{\mathrm{S_3}}}{\partial{\mathrm{W_s}}}+
\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{O_3}}}\frac{\partial{\mathrm{O_3}}}{\partial{\mathrm{S_3}}}\frac{\partial{\mathrm{S_3}}}{\partial{\mathrm{S_2}}}\frac{\partial{\mathrm{S_2}}}{\partial{\mathrm{W_s}}}+
\frac{\partial{\mathrm{L_3}}}{\partial{\mathrm{O_3}}}\frac{\partial{\mathrm{O_3}}}{\partial{\mathrm{S_3}}}\frac{\partial{\mathrm{S_3}}}{\partial{\mathrm{S_2}}}\frac{\partial{\mathrm{S_2}}}{\partial{\mathrm{S_1}}}\frac{\partial{\mathrm{S_1}}}{\partial{\mathrm{W_s}}}
\end{align}

 **注意：在上面求$W_x$偏导的表达式中，从左往右的$W_x$分别表示的是t=3,t=2,t=1时刻下的权重信息，$W_s$同理**

&ensp; &ensp;从上面的求导过程中，我们在t3时刻对$W_x,W_s$求偏导的过程中，会不断的使用到t2和t1时刻的$W_x,W_s$权重信息，即随着时间序列会产生**长期依赖**,一旦时间序列变得较长的情况下，那么对$W_x,W_s$的计算将变的非常的冗长，由于$W_x,W_s$计算过程十分相似，于是以$W_x$为例归纳权重矩阵的偏导计算公式：

\begin{align}
\frac{\partial{\mathrm{L}_{t}}}{\partial{\mathrm{W}_{x}}}=\sum_{k=0}^{t}\frac{\partial{\mathrm{L_t}}}{\partial{\mathrm{O_t}}}
\frac{\partial{\mathrm{O_t}}}{\partial{\mathrm{S_t}}}(\prod_{j=k+1}^{t}\frac{\partial{\mathrm{S_j}}}{\partial{\mathrm{S}_{j-1}}})\frac{\partial{\mathrm{S_k}}}{\partial{\mathrm{W_x}}}\\
\end{align}

\begin{align}
\frac{\partial{\mathrm{Sj}}}{\partial{\mathrm{S_j-1}}}=tanh'* W_x
\end{align}

&ensp; &ensp;通过上面的两个归纳的公式可以看到，在对$W_x$进行求偏导的过程中，$W_x$将会以指数程度进行增长，其表现形式为$(W_x)^{t-k}$,当t变得非常大（靠后的时间序列），而k变的非常小的时候（靠前的时间序列），**t-k将变得非常大**，而我们知道权重矩阵在初始化时是随机分配的，当$W_x < 1$时就会出现梯度消失的情况，而当$W_x > 1$时就会出现梯度爆炸的情况。针对RNN存在的这种缺陷，于是提出了RNN的变体，LSTM。

### 2.2 LSTM模型与RNN联系与区别

&ensp; &ensp;**LSTM(Long-short-term-Memory 长短时记忆网路)** 是RNN模型的一种变体，通过上面的分析我们知道RNN只包含了一个隐藏层状态h,并且仅仅**对于短期记忆敏感**，而容易丢失长期记忆，因此我们增加一个状态来保存长期记忆。LSTM 通过加入**细胞状态（Cell State）和门控思想(gate)** 对过往的信息进行选择的记忆。

&ensp; &ensp;和RNN模型进行比较理解，RNN得到的**hidden_state**是上一层的**hidden_state**和当前**input**信息的**直接**向量运算。简单而直接的保留了**全部的信息**，随着序列的不断加长，循环的不断进行，之前的信息可能会发生**遗忘**；而LSTM模型在RNN的基础上加入**细胞状态（Cell State）和门控思想(gate)**，这些都是基于上一层**hidden_state**，通过很多**间接**操作得到当前**hidden_state**，这样就可以选择哪些信息**被保留**而传入下一个循环，哪些信息直接被**主动遗忘**，达到筛选和保留重要信息的作用。

**RNN:**

<div style="width:680px;height:350px;margin:0 auto;">
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png"/>
</div>

**LSTM:**

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

&ensp; &ensp;将上面的LSTM模型和RNN模型对比,可以发现不同的是状态信息从**单通道输入变成了双通道输入**，并且**增添了许多门（gate）和运算**来对输入信息进行处理，即对信息进行有选择性的记忆。下面将对上图涉及到的门和运算一一解释。

### 2.3 LSTM模型详解

#### 2.3.1 门思想和符号介绍
&ensp; &ensp;**门思想**：门实际上就是一层全连接层，它的输入是一个向量，输出是一个0到1的实数向量。门思想类似于过滤器，对于输入的信息进行过滤和选择，这里用到的是sigmoid函数，sigmoid函数的输出介于0-1,表示对于输入的数据按照怎样的比率通过该道门。**如果得到的值为0表示任何内容都不通过该门，如果得到的值为1表示所有的内容通过该门**<br/>

![](https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fimg-blog.csdnimg.cn%2Fimg_convert%2F668ea6da04c5bfbdda096a6770efc9e6.png&refer=http%3A%2F%2Fimg-blog.csdnimg.cn&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=auto?sec=1670604237&t=ec8b6ac44865fd59b4a3303402c8eafb)

**符号介绍**：<br/>
* $X_t$表示在t时刻输入的信息
* ${h}_{t-1}$表示在t-1时刻的状态信息
* ${C}_{t-1}$表示在t-1时刻的细胞残留的信息
* ${σ、tanh}$的黄色方框，表示的是神经网络层，
* ${×}$的圆圈表示的是哈达玛积，运算规则为对应的位置元素相乘
* ${+}$表示的是向量相加
* ${f_t}$表示的是遗忘门，可以理解为以怎样的比例来遗忘之前的信息
* ${i_t}$表示的是输入门，可以理解为对于当前的输入信息以怎样的比例来保留
* ${\tilde{C_t}}$可以理解为对输入内容创建候选值向量，其中的东西就是最后加入细胞状态的内容
* ${O_t}$表示的是输出门，可以理解为以怎样的比例将当前的值进行输出

#### **遗忘门（${f_t}$）**
![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png)
&ensp; &ensp;遗忘门通过查看**当前**的输入以及**之前**的状态信息，通过遗忘门的权重矩阵${W_f}$，乘上当前的输入矩阵$[{h}_{t-1},{x_t}]$，经过sigmoid函数得到每一个输入的**遗忘比例**，然后会在之后将输入的**遗忘比例**作用到${C}_{t-1}$上，决定了上一时刻的单元状态${C}_{t-1}$有多少内容**保留**到当前时刻${C_t}$。如果输入项的维度是${d_x}$,隐藏层的维度是${d_h}$,单元状态的维度是${d_c}$(通常${d_c}$ = ${d_h}$)，则输入矩阵的维度为$({d_h}$ + ${d_x})*$${d_c}$,权重矩阵${W_f}$由两部分构成，分别是$W_{fh}$和$W_{fx}$,其中$W_{fh}$的维度是${d_c}*{d_h}$,$W_{fx}$的维度是${d_c}*{d_x}$,因此${W_f}$的维度是${d_c}*$$({d_h} + {d_x})$

#### **输入门（${i_t}$）**
![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png)
&ensp; &ensp;输入门与以遗忘门的作用原理比较类似，通过${W_i}$乘上当前的输入矩阵，经过sigmoid函数得到一个输入的**保留比例**，${\tilde{C_t}}$描述的是当前的输入状态，通过tanh函数得到一个候选向量（实际要加入到细胞状态的内容，类似于**数据拷贝**，但是不是简单的复制，而是经过tanh作用），之后将**保存的比例**，乘上候选向量得到要新加入的内容。

#### **细胞状态的更新**
![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png)
&ensp; &ensp;通过之前的解释，这里就比较好理解了，将t-1时刻的细胞状态(Cell State)中的部分内容**遗忘**，再加入t时刻经过**筛选**的新的内容，从而得到了t时刻的细胞状态${C_t}$。这样就将**长期的记忆**${C}_{t-1}$与**当前的记忆**${\tilde{C_t}}$组合在了一起，形成了新的细胞状态。由于**遗忘门**的控制，它可以保存很久很久之前的信息，由于**输入门**的控制，它又可以避免当前无关紧要的内容进入记忆。

#### **输出门（${O_t}$)**

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png)
&ensp; &ensp;与前面同理，输出门计算中要输出内容的比例${O_t}$，控制了**长期记忆**对当前输出的影响，表示要输出${C_t}$的哪些部分，然后将经过tanh变化的候选向量乘上对应的**输出比例**得到最终的输出。

### 2.4 LSTM代码实现

#### 2.4.1 LSTM代码拆分版

&ensp; &ensp;LSTM的代码本质上就是在一个RNN循环神经单元基础上(建议再回看一下RNN的代码拆分，与LSTM的代码对比加强理解)，加入**细胞状态（Cell State）和遗忘门，输入门，输出门**来选择性的将信息存入hidden_state。用公式来表示的话就是：

<center><img src="LSTM公式.jpeg" width=60%><center>

&ensp; &ensp;那么我们可以通过构建一个`LSTMstep`方法(LSTM的前向传播)来实现上述公式：

In [10]:
import numpy as np
import torch
import torch.nn as nn

def LSTMstep(input,init_states,w_ih,w_hh,b_ih,b_hh):
    # 从传递过来的init_states中获取初始状态，如果没有设定，PyTorch会默认填充相同维度的零向量
    # w_ih,w_hh,b_ih,b_hh为初始权重矩阵
    h0, c0 = init_states # 定义的初始状态，即初始时刻下的细胞状态和隐藏状态
    bs, T, i_size = input.shape
    # 注意这里的w_ih的维度，由于四个门的运算相同，i（输入门）,f（遗忘门）,g（细胞内容）,o（输出门）
    # 因此为了方便运算，将四个权重矩阵进行堆叠，统一和x进行矩阵运算，运算完成再将运算结果进行拆分
    h_size = w_ih.shape[0] // 4 # 拆分出h_size的值
    
    hidden_state = h0  #当前隐藏状态
    cell_state = c0   #当前细胞状态
    
    #对权重矩阵进行扩围，加入批处理的维度信息。然后用于矩阵乘法运算
    batch_w_ih = w_ih.unsqueeze(0).tile(bs, 1, 1) # 维度由(4*h_size,i_size) -> (bs, 4*h_size, i_size)
    batch_w_hh = w_hh.unsqueeze(0).tile(bs, 1, 1) # 维度由(4*h_size,h_size) -> (bs, 4*h_size, h_size)
    
    output_size = h_size 
    output = torch.zeros(bs, T, output_size) # 因为是最后一层隐藏层的的输出，所以维度是h_size
    
    #根据序列循环运算
    for t in range(T):        
        x = input[:, t, :] # 取出t时刻(t序列)的输入，维度为bs * i_size 
        w_times_x = torch.bmm(batch_w_ih, x.unsqueeze(2)) # bmm是含batch的矩阵乘法，这里对x进行了扩围，x的原始维度维度为bs * i_size ,扩容后的维度为bs * i_size * 1，运算后的维度为(bs, 4*h_size, 1)
        w_times_x = w_times_x.squeeze(-1) # 这里是删掉最后一个维度，维度由(bs, 4*h_size, 1)变为(bs, 4*h_size)
        
        w_times_h = torch.bmm(batch_w_hh, hidden_state.unsqueeze(2)) 
        w_times_h = w_times_h.squeeze(-1)
        
       
        
        # 分别计算输入门（i）、遗忘门（f）、cell门（g）、输出门（o），由于之前矩阵运算是把四个权重矩阵拼接起来计算的，这里使用切片运算得到各自的结果代入门限函数
        i_t = torch.sigmoid(w_times_x[:, :h_size] + b_ih[:h_size] + \
                      w_times_h[:, :h_size] + b_hh[:h_size])
        f_t = torch.sigmoid(w_times_x[:, h_size:2 * h_size] + b_ih[h_size:2 * h_size] + \
                      w_times_h[:, h_size:2 * h_size] + b_hh[h_size:2 * h_size])
        g_t = torch.tanh(w_times_x[:, 2 * h_size:3 * h_size] + b_ih[2 * h_size:3 * h_size] + \
                   w_times_h[:, 2 * h_size:3 * h_size] + b_hh[2 * h_size:3 * h_size])
        o_t = torch.sigmoid(w_times_x[:, 3 * h_size:] + b_ih[3 * h_size:] + \
                      w_times_h[:, 3 * h_size:] + b_hh[3 * h_size:])
                            
        # 计算ct和ht
        cell_state = f_t * cell_state + i_t * g_t
        hidden_state = o_t * torch.tanh(cell_state)
                            
        output[:, t, :] = hidden_state
    return output,(hidden_state, cell_state)

&ensp; &ensp;上面的代码块相较于的`RNNstep`封装的更为完整，最开始的部分为**输入文本向量**、**生成的权重矩阵**、**隐藏状态**和**细胞状态向量**，并通过降维升维等python向量运算进行向量对齐；主体为**输入门（i_t）**、**遗忘门（f_t）**、**cell门（g_t）**、**输出门（o_t）**计算公式的实现；进而利用for循环更新隐藏状态。

给定和上一章RNN相同的输入文本数据“我 爱 西 财”来进行实验

In [11]:
import numpy as np
import torch
import torch.nn as nn

text_data = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])

batch_size = 1 #只有一句话“我爱西财”

input_text = torch.from_numpy(text_data[np.newaxis, :]).float()  # np.newaxis矩阵扩充维度，即加入新维度，batch_size = 1
print(f"input.shape为:{input_text.shape}")  #  [seq_len * input_size]
seq_len = input_text.shape[1]
print(f"seq_len为:{seq_len}")

input.shape为:torch.Size([1, 4, 4])
seq_len为:4


**随机初始化隐藏状态和细胞状态；随机初始化权重矩阵**

In [12]:
h_size = 5  #输出张量维度，即定义的隐藏层维度

h0 = torch.randn(batch_size, h_size) # 初始的隐藏层状态
c0 = torch.randn(batch_size, h_size) # 初始的细胞状态

weight_ih = torch.randn(20, 4)  #[4 * hidden_size,input_size]
weight_hh = torch.randn(20, 5)  #[4 * hidden_size,hidden_size]
bias_ih = torch.randn(20)   # 4 * hidden_size
bias_hh = torch.randn(20)  # 4 * hidden_size

**这里特别说明一下维度20**，因为LSTM中四个门的运算非常类似，所以为了加快运算，**将四个门的权重矩阵做了堆叠**。weight_ih包含了$w_{ii}$,$w_{if}$,$w_{ig}$,$w_{io}$，weight_hh也是同理。本质上维度表示为**[4 * hidden_size,input_size]**与**[4 * hidden_size,hidden_size]**

In [13]:
output,(h_final,c_final) = LSTMstep(input_text,(h0, c0), weight_ih, weight_hh, bias_ih, bias_hh)

print('\n 自实现lstm的 output：')
print(output)


 自实现lstm的 output：
tensor([[[-0.9052, -0.0063, -0.0673,  0.0019, -0.7088],
         [-0.6255, -0.2410, -0.0029,  0.0105, -0.6423],
         [-0.1541, -0.1126,  0.0597,  0.1839, -0.3486],
         [-0.5726, -0.6565,  0.2462,  0.2883, -0.2416]]])


#### 2.4.2 LSTM代码pytorch实现

&ensp; &ensp;为了更好的理解LSTM的前向传播过程，下面我们会再调用PyTorch官方的API有一个直观的理解,然后我们自己再实现LSTM的前向传播过程forward函数加深理解。下面是LSTM API接口的链接，这里仅以**单层单向**的LSTM作为实例，感兴趣的同学可以自行实现多层双向的LSTM。[LSTM API的链接](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

&ensp; &ensp;下面展示的是Pytorch中的LSTM API,对于其中的常用的超参数做一个简单说明。**input_size**表示的是输入特征的维度，比如我们在后面用到的字符预测，那么input_size的维度就是字典长度大小。**hidden_size**是表示自定义的隐藏层单元个数。**num_layers**表示的是LSTM的层数，在我们的实验中，num_layers = 1。**batch_first**是用来调整输入张量和输出张量的维度顺序。即第0维是batch还是seq,batch是指我们一次处理的样本个数，而seq则是表示的是时间序列长度，因为LSTM也是一个关于时间序列的模型。

In [14]:
# "我 爱 西 财"的文本输入
text_data = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
batch_size = 1 #只有一句话“我爱西财”
input_text = torch.from_numpy(text_data[np.newaxis, :]).float()  # np.newaxis矩阵扩充维度，即加入新维度，batch_size = 1

# 设定初始隐藏层状态和细胞状态
h_size = 5  #hidden_size
i_size = input_text.shape[1]

h0 = torch.randn(batch_size, h_size) # 初始的隐藏层状态
c0 = torch.randn(batch_size, h_size) # 初始的细胞状态

# 调用官方的LSTM API
lstm_layer = nn.LSTM(i_size, h_size, batch_first = True) # 得到一个LSTM 对象模型
output,(h_final, c_final) = lstm_layer(input_text, (h0.unsqueeze(0), c0.unsqueeze(0))) #这里扩维的目的是遵循pyTorch中的LSTM API数据维度，数据维度由(bs, h_size) 变为（1,bs,h_size）
print('\n pytorch lstm的 output：')
print(output)


 pytorch lstm的 output：
tensor([[[-0.3419, -0.1862, -0.1406, -0.0967,  0.1691],
         [-0.1943,  0.0059, -0.1500, -0.0451,  0.1897],
         [-0.1982,  0.0891, -0.1605, -0.0093,  0.1095],
         [-0.1580,  0.1884, -0.1376,  0.0375,  0.1837]]],
       grad_fn=<TransposeBackward0>)


&ensp; &ensp;对LSTM的输入进行一个简单说明。可以看到LSTM的输入包含两个部分，第一部分是一个**input张量**，其维度为**(seq,input_size)**或者**(seq,bs,input_size)**，当超参数**batch_first = True**，其维度调整为**(bs,seq,input_size)**。第二部分输入是一个元组**(h_0,c_0)**，表示的初始时刻下的细胞状态和隐藏状态，可以看到它的维度与LSTM的层数和单/双向有关，由于本次使用的是单层单向的LSTM，所以h_0,c_0的维度都为**（1，hidden_size）**。

## 3.利用循环神经网络实现文本生成

**本次实验内容为给定一段文本，让循环神经网络依次从左到右读入这句话，并在文字末尾选择一个最可能的新出现的字填入，即完成NLP中的文本生成任务**

In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

**给定语料库**

In [16]:
text = '西南财经大学金融科技国际联合实验室是一个以国内外顶尖高校，国际金融机构、政府以及监管机构多方联合建设的开放性网格状资源集聚平台，' \
        '是在金融领域内通过与业界的紧密合作及重大项目攻关，研发金融科技解决方案和服务的创新平台。'\
        '是一个以国内外顶尖高校，国际金融机构、政府以及监管机构多方联合建设的开放性网格状资源集聚平台，'\
        '是在金融领域内通过与业界的紧密合作及重大项目攻关，研发金融科技解决方案和服务的创新平台。'

**训练文本**

In [17]:
seq_data = ['西南财经大学金融科技国际联合实验室', 
            '是在金融领域内通过与业界的紧密合作及重大项目攻关，研发金融科技解决方案和服务的创新平台', 
            '国内外顶尖高校，国际金融机构、政府以及监管机构多方联合建设',
            '金融领域内通过与业界的紧密合作及重大项目攻关，研发金融科技解决方案和服务']

**获得训练文本中最长的句子**

In [18]:
len_seq = []
for seq in seq_data:
    seq_len = len(seq)
    len_seq.append(seq_len)
max_len = max(len_seq)

print(f"最长句子字数为：{max_len}")

最长句子字数为：43


**根据语料库生成数字-文本相对应的词典**

In [19]:
char_list = [c for c in text]  #生成list保存字符

# 完成字母到数字的映射(在之前的章节有过介绍并展示过代码)
word_dict = {w:i for i,w in enumerate(set(char_list))}  # 利用python集合的定义去重
# 完成数字到字母的映射
number_dict = {i:w for i,w in enumerate(set(char_list))}

print(number_dict)  # 得到数字与文字相对应的字典

{0: '校', 1: '决', 2: '放', 3: '西', 4: '紧', 5: '案', 6: '攻', 7: '南', 8: '网', 9: '技', 10: '密', 11: '解', 12: '重', 13: '财', 14: '作', 15: '以', 16: '目', 17: '通', 18: '监', 19: '管', 20: '设', 21: '实', 22: '顶', 23: '聚', 24: '集', 25: '、', 26: '方', 27: '际', 28: '台', 29: '联', 30: '外', 31: '的', 32: '室', 33: '及', 34: '尖', 35: '多', 36: '过', 37: '关', 38: '国', 39: '研', 40: '服', 41: '融', 42: '与', 43: '学', 44: '务', 45: '发', 46: '性', 47: '验', 48: '领', 49: '构', 50: '是', 51: '。', 52: '个', 53: '内', 54: '域', 55: '一', 56: '格', 57: '科', 58: '开', 59: '高', 60: '项', 61: '创', 62: '在', 63: '资', 64: '府', 65: '机', 66: '状', 67: '和', 68: '金', 69: '政', 70: '建', 71: '界', 72: '业', 73: '平', 74: '，', 75: '合', 76: '大', 77: '源', 78: '新', 79: '经'}


**根据词典完成seq_data的文本向量化，并把每个句子中最后一个字截取出来作为模型需要预测的Y,句子中剩余文本即为X**

In [20]:
input_data,target_data = [],[]
n_class = len(word_dict)
for seq in seq_data:
    input_id = [word_dict[n] for n in seq[:-1]]  # 获取输入字母在字母表中的索引 
    input_id = np.pad(input_id,(0,max_len-1-len(input_id)))
    target = word_dict[seq[-1]] # 获取目标字母在字母表中的索引
    input_data.append(np.eye(n_class)[input_id]) # 将input列表中的每个元素转换为一个n_class(26)维的one-hot向量，并将该向量添加到input-data中
    target_data.append(target)
print(target_data[:5])

[32, 28, 20, 44]


**转化为torch.Tensor格式**

In [21]:
input_data = torch.FloatTensor(input_data)
target_data = torch.LongTensor(target_data)

**定义模型**

In [22]:
# 定义每个cell隐藏层的神经元个数
n_hidden = 128

# 定义文本预测的LSTM的模型
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size = n_class,hidden_size = n_hidden) # pytorch nn.LSTM：n_class * n_hidden * 1(1代表只有1层LSTM) 
        self.W = nn.Linear(n_hidden,n_class,bias = False)  # 声明了一个线性层，对输入的内容进行线性变换 y = x*w + b ,可以近似的理解为128 * 26的权重矩阵
        self.b = nn.Parameter(torch.ones(n_class)) # 申明了26个偏置项，每个偏置项的内容为1
        
    # 前向传播
    def forward(self,X):
        input = X.transpose(0, 1) # 变换了张量的维度，X原来的维度是 10 * 3 * 26，变换维度后为 3 * 10 * 26
        #初始化隐藏态状态和细胞状态
        hidden_state = torch.zeros(1,len(X),n_hidden) 
        cell_state = torch.zeros(1,len(X), n_hidden) 
        outputs,(_,_) = self.lstm(input,(hidden_state,cell_state))
        outputs = outputs[-1] # batch * n_hidden(10 * 128)
        model = self.W(outputs) + self.b
        return model
    

In [23]:
model = TextLSTM() #得到一个训练模型对象
criterion = nn.CrossEntropyLoss()  # 使用交叉熵对训练误差进行估计
optimzer = optim.Adam(model.parameters(),lr = 0.001) # 定义优化器，学习率为0.001

**模型训练**

In [24]:
epoch = 600
for i in range(epoch):
    #每次训练前将梯度变量置为0
    optimzer.zero_grad()

    output = model(input_data)
    loss = criterion(output,target_data)
    if(i + 1) % 100 == 0:
        print('Epoch:', '%04d' % (i + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimzer.step()

Epoch: 0100 cost = 1.102687
Epoch: 0200 cost = 0.696015
Epoch: 0300 cost = 0.173820
Epoch: 0400 cost = 0.033342
Epoch: 0500 cost = 0.016281
Epoch: 0600 cost = 0.009861


**利用训练好的模型预测句子后面应该接什么词**

In [25]:
pretext = '西南财经大学'  #给定需要生成的文本

pre_id = [word_dict[n] for n in pretext]   # 文本向量化

pre_data = np.eye(n_class)[pre_id]  
pre_data = torch.FloatTensor(pre_data)
# 转化为模型需要的输入格式

predict = model(torch.unsqueeze(pre_data,0)).data.max(1, keepdim=True)[1] 
# 括号里面的1表示得到二维(一行中)中的最大值，keepdim = True时，表示将维度修改为1,后面的[1]表示取的是索引值
# 获得预测的向量

number_dict[predict.squeeze().item()]
print(pretext, '->', number_dict[predict.squeeze().item()])

西南财经大学 -> 台


**而我们把预测的内容接到句子后面，循环这个步骤，那么我们就可以不断生成文字，这就是文本生成最本质的内容：在文本后面与生成概率最大的一个字。当我们不断优化我们的模型，增加训练语料，生成的效果就会逐渐提升，例如现在文本生成的一个非常火热的模型chatGPT,已经可以完成一些非常有创造力的生成任务，我们在后面的章节也会复现我们的GPT系列模型**